## Fit per onde e ottica ##
Costruiamo un fit con il metodo della $\sigma_{\mathrm{fit}}$. Precisiamo che la funzione dovrà prendere un array NUMPY con due colonne, la prima delle x e la seconda delle y.

In [1]:
import numpy as np

In [2]:
def lin_fit_sigma_fit(x,y):
    N = len(x)
    x2 = x**2
    xy = x*y
    X = sum(x)
    X2 = sum(x2)
    XY = sum(xy)
    Y = sum(y)
    A = np.array([[X2, X],[X, N]])
    B = np.array([[XY],[Y]])
    sol = np.linalg.solve(A, B)
    a = sol[0, 0]
    b = sol[1, 0]
    Delta = N*X2-X**2
    y_fit = a*x+b
    sigma_fit = np.sqrt(sum((y_fit-y)**2)/(N-1))
    sigmaa = sigma_fit * np.sqrt(N/Delta)
    sigmab = sigma_fit * np.sqrt(X2/Delta)
    covab = -X/Delta * sigma_fit**2
    rho = covab/(sigmaa*sigmab)
    res = np.array([a, sigmaa, b, sigmab, rho])
    return res

In [8]:
def plot(x,y,xe,ye,fit=False,cs=5):
    fig, ax = plt.subplots(figsize=(10,10), dpi=100)
    ax.errorbar(x,y,xerr=xe,yerr=ye,linestyle='',capsize=cs,fmt='.',label='data')
    if fit==True:
        a,sigmaa,b,sigmab,rho=lin_fit_sigma_fit(x,y)
        def lin(x,a,b):
            return a*x+b
        m,M=x.min(),x.max()
        ax.plot([m,M],[lin(m,a,b),lin(M,a,b)],zorder=100,color='orange',label=r'fit $a\cdot x+b$:'+'\n'+r'a={:.4f}+/-{:.4f}'.format(a,sigmaa)+'\n'+'b={:.4f}+/-{:.4f}'.format(b,sigmab)+'\n'+'corr.:{:.1f}'.format(rho))    
    ax.legend()
    plt.show()
    print((table(x,y,xe,ye).sort_values(by=['x'])))

In [9]:
def labhist(alpha1, w=1):
    fig,ax=plt.subplots(figsize=(10,10),dpi=125)
    l=len(alpha1)
    sigma1=alpha1.std(ddof=1)
    mi1=alpha1.mean()
    U=np.array([str(a) for a in np.unique(alpha1)], dtype=str)
    freqs=np.array([str(len(alpha1[alpha1==a])) for a in np.unique(alpha1)],dtype=str)
    Title=np.array([a+'  '+b for a,b in zip(U, freqs)])
    ax.bar(alpha1, [len(alpha1[alpha1==a]) for a in alpha1],width=w, label=r'$\sigma=$'+'{:.1f}°'.format(sigma1)+'\n'+r'$\mu=$'+'{:.1f}°'.format(mi1))
    ax.legend(title=r'$x$  N'+'\n'+'\n'.join(Title), title_fontsize=10)
    ax.set_ylabel(r'N')
    ax.set_xticks(alpha1)
    ax.set_xticklabels(alpha1,rotation=60)
    plt.show()
    return (mi1, sigma1)

In [5]:
def wm(sample,sigmas):
    w=np.power(sigmas,-2)
    c=(sample*w).sum()/w.sum()
    s=np.power(1/w.sum(),0.5)
    return (c,s)

In [6]:
def round_sig(x, sig=2):
    from math import floor, log10
    return round(x, sig-int(floor(log10(abs(x))))-1)
rs=np.vectorize(round_sig)

In [7]:
def round_err_based(value,error):
    from math import log10,ceil,floor
    v=(floor(log10(abs(value))))
    e=(floor(log10(abs(error))))
    sig=(v-e)+1
    return round_sig(value,sig)
reb=np.vectorize(round_err_based)

In [9]:
def max_err_prop_prod(x,ex,y,ey):
    return x*ey+ex*y
def max_err_prop_div(x,ex,y,ey):
    return 1/y*ex+x/y**2*ey
mepp=np.vectorize(max_err_prop_prod)
mepd=np.vectorize(max_err_prop_div)

In [17]:
import pandas as pd
def table(x,y,ex=None,ey=None,bx=True,by=True):
    if bx==True and by==True:
        dic={'x':x,'ex':ex,'y':y,'ey':ey}
    elif bx==False:
        dic={'x':x,'y':y,'ey':ey}
    elif by==False:
        dic={'x':x,'ex':ex,'y':y}
    else:
        dic={'x':x,'y':y}
    tab=pd.DataFrame(dic)
    return (tab)

def tab(names,cols):
    dic={}
    for n,c in zip(names,cols):
        dic[n]=c
    df=pd.DataFrame(dic)
    return df

In [3]:
import pandas as pd
def QY(qm,ym,qM,yM):
    errq=rs((qM-qm)/2,1)
    erry=rs((yM-ym)/2,1)
    y=reb((yM+yM)/2,erry)
    q=reb((qM+qm)/2,errq)
    dic={'q':q,'errq':errq,'y':y,'erry':erry}
    tab=pd.DataFrame(dic)
    return tab